In [1]:
from datamodules.mt_distillation_datamodule import MTDistillationDatamodule

dm = MTDistillationDatamodule(batch_size=5)
dm.setup()
batch = next(iter(dm.train_dataloader()))
batch

Using custom data configuration din0s--ccmatrix_en-ro-b5eb1c3b243f258f
Reusing dataset parquet (/home/angelos/.cache/huggingface/datasets/din0s___parquet/din0s--ccmatrix_en-ro-b5eb1c3b243f258f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/angelos/.cache/huggingface/datasets/din0s___parquet/din0s--ccmatrix_en-ro-b5eb1c3b243f258f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e04cb05df3d5e9e2.arrow
Using custom data configuration j0hngou--ccmatrix_en-fr-c75b778ef0fed3ea
Reusing dataset parquet (/home/angelos/.cache/huggingface/datasets/j0hngou___parquet/j0hngou--ccmatrix_en-fr-c75b778ef0fed3ea/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/angelos/.cache/huggingface/datasets/j0hngou___parquet/j0hngou--ccmatrix_en-fr-c75b778ef0fed3ea/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-0574ddf6b3bda371.arrow


{'en-ro': {'source': tensor([[13959,  1566,    12,  3871,    29,    10,    12,   627,  3823, 14743,
               5,     1,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0]]),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
  'target': tensor([[  325,     3,  7741,  3630,   627, 28738,    23, 22281,    35,    15,
               1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,    

In [2]:
print(batch["en-fr"]["source"].shape)
print(batch["en-ro"]["source"].shape)

torch.Size([4, 43])
torch.Size([1, 43])


In [3]:
from transformers import AutoModelForSeq2SeqLM

model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [6]:
model(input_ids=batch["en-ro"]["source"], 
      attention_mask=batch["en-ro"]["attention_mask"], 
      decoder_input_ids=batch["en-ro"]["target"],     
      decoder_attention_mask=batch["en-ro"]["decoder_attention_mask"]
     ).logits

tensor([[[-21.9249,  -8.9075,  -9.3580,  ..., -46.9724, -47.1455, -47.0554],
         [-21.4823, -10.0551,  -4.1233,  ..., -42.9394, -43.0076, -42.9269],
         [-23.4109,  -9.6241, -13.6325,  ..., -47.0220, -47.0796, -47.0454],
         ...,
         [-10.6927,  -4.4714,  -9.4419,  ..., -39.3290, -39.4041, -39.4781],
         [-10.8941,  -4.4819,  -9.5339,  ..., -39.4958, -39.5699, -39.6404],
         [-10.9117,  -4.4439,  -9.5682,  ..., -39.4095, -39.4821, -39.5508]]],
       grad_fn=<UnsafeViewBackward0>)